In [2]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'
data_path= 'C:/Users/minkyu/Desktop/open/'
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [5]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [7]:
from sentence_transformers import SentenceTransformer
import torch
# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Embedding Vector 추출 모델을 GPU로 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False).to(device)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [8]:
grouped = train.groupby(["공종(중분류)", "인적사고"])

res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    
    if len(plan) < 2:  # 데이터가 1개 이하라면 유사도 계산 불가
        continue  

    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    
    # 가장 평균 유사도가 높은 문장을 선택
    best_idx = similarity.mean(axis=1).argmax()
    cosine_res += similarity[best_idx].tolist()
    res[name] = plan.iloc[best_idx]

arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")


  0%|          | 0/753 [00:00<?, ?it/s]

 97%|█████████▋| 733/753 [03:49<00:06,  3.19it/s]

Range 0.0 - 0.1: 0개
Range 0.1 - 0.2: 28개
Range 0.2 - 0.3: 183개
Range 0.3 - 0.4: 706개
Range 0.4 - 0.5: 1905개
Range 0.5 - 0.6: 4241개
Range 0.6 - 0.7: 7092개
Range 0.7 - 0.8: 6539개
Range 0.8 - 0.9: 1870개
Range 0.9 - 1.0: 509개


In [27]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [ ]:
sample.to_csv("baseline.csv", index=False, encoding='utf-8-sig')

In [35]:
from tqdm import tqdm


# sample 데이터프레임에 값 할당 (진행 상태 표시)
for i in tqdm(range(len(test)), desc="Processing", ncols=100):  # tqdm을 사용하여 진행 상태 표시
    accident = test.loc[i, "인적사고"]  # 사고 유형
    category = test.loc[i, "공종(중분류)"]  # 공종(중분류) 값

    # (공종, 인적사고) 조합을 기준으로 res에서 재발방지대책을 가져옴
    key = (category, accident)
    if key in res:  # 해당 조합이 res에 존재하는 경우
        sample.loc[i, "재발방지대책 및 향후조치계획"] = res[key]  # 문장 할당
        
        # 벡터 값도 가져와서 할당
        vector = res_v[key]
        sample.iloc[i, 2:2+len(vector)] = vector  # 벡터 길이에 맞게 할당
        
        # 진행 상황을 출력
        if i % 100 == 0:  # 100번째마다 진행 상황 출력
            print(f"Processing {i}/{len(test)} - Accident: {accident}, Category: {category}")


Processing:   0%|                                                   | 1/964 [00:00<12:40,  1.27it/s]

Processing 0/964 - Accident: 부딪힘, Category: 철근콘크리트공사


Processing:  10%|█████▏                                           | 101/964 [01:17<11:05,  1.30it/s]

Processing 100/964 - Accident: 절단, 베임, Category: 철근콘크리트공사


Processing:  21%|██████████▎                                      | 204/964 [02:36<09:33,  1.33it/s]

...

Processing:  83%|████████████████████████████████████████▋        | 801/964 [10:11<02:05,  1.30it/s]

Processing 800/964 - Accident: 끼임, Category: 관공사 부대공사


Processing:  93%|█████████████████████████████████████████████▊   | 901/964 [11:28<00:48,  1.30it/s]

Processing 900/964 - Accident: 넘어짐(미끄러짐), Category: 방수공사


Processing: 100%|█████████████████████████████████████████████████| 964/964 [12:17<00:00,  1.31it/s]


In [38]:
# 주어진 문장
sentence = "안전사고 재발 방지를 위한 작업자 교육 및 안전보호구 착용 철저, 안전계단 설치 경사 재조정 및 안전교육 실시."

# 문장의 임베딩 계산
embedding = model.encode(sentence)

# 결과 출력 (임베딩 벡터)
print(embedding)

[-4.78118896e-01  7.04488814e-01 -3.63128513e-01  2.94981360e-01
 -5.42469859e-01  7.25417674e-01  4.43165392e-01 -5.55303276e-01
  8.93017799e-02  5.49925208e-01 -2.59051938e-02  4.91817184e-02
  3.52360994e-01  4.10469286e-02 -1.22694664e-01 -2.40604609e-01
  5.89486778e-01 -6.27028704e-01 -6.92812681e-01  7.07320392e-01
  5.79684496e-01  1.59163535e-01  5.04967809e-01 -7.26567566e-01
  2.30825037e-01  9.79350805e-01 -3.80390286e-01  6.22996271e-01
 -3.06614768e-02  4.83825147e-01  4.39409047e-01  2.09974840e-01
  3.91219705e-01 -1.00491130e+00  6.62092566e-01 -5.64460039e-01
 -1.91128835e-01  1.30840331e-01  2.05648586e-01 -3.52971524e-01
  3.73402834e-01  5.51174998e-01 -7.20825419e-02  1.16354120e+00
 -9.87309277e-01  1.33319116e+00 -7.74600744e-01  5.85672334e-02
  4.91220593e-01 -1.12304783e+00  9.90402579e-01  4.55773115e-01
  5.46406746e-01  4.00002927e-01  1.53589118e+00  4.97605503e-01
 -1.76906991e+00 -6.10031545e-01  4.60268945e-01 -2.42013678e-01
 -1.28970565e-02 -2.33105

In [41]:
sample.iloc[42, 2:2+len(vector)] = embedding

In [46]:
sample.iloc[160, 2:2+len(vector)]= embedding

In [47]:
sample.to_csv(base_path+"/rule_base_v2.csv", index=False, encoding='utf-8-sig')